# 2.4.2. Normal Distribution

### Learning Objectives:
- [Normal Distribution](#Normal-Distribution)
- [Central Limit Theorem](#Central-Limit-Theorem)
- [Standardizaton](#Standardization)

# Normal Distribution

The __normal distribution__ is a continuous probability distribution that is commonly used to approximate the probability distribution of continuous random varaibles whose true distributions are not known. Given its shape, it is also informally referred to as a __bell curve__, although there are other less common bell-shaped distributions out there. It has a probability density function given by the following function:

$$f(x) = \frac{1}{\sigma \sqrt{2\pi}}e^{-\frac{1}{2}(\frac{x-\mu}{\sigma})^{2}}$$

Where $\sigma$ is the population standard deviation and $\mu$ the population mean of the continuous random variable x. We generally denote a normally distributed random variable as shown below:

$$X \sim N(\mu, \sigma ^{2})$$

We can create a Python function to model it as follows:

In [1]:
import numpy as np
import plotly.graph_objects as go
import utils

def class_heights(n):
    return np.random.normal(175, 4, n)

def normal(x, mean, std):
    exponent = ((x-mean)/std)**2
    exponent *= -0.5
    output = np.exp(exponent)/(std*np.sqrt(2*np.pi))
    return output

In [2]:
mean = 12
std_store = [0.1, 0.5, 1, 2]
x_vals = np.linspace(8, 16, 1000)
colors = ["orange", "red", "blue", "green"]
fig = go.Figure()
for std, color in zip(std_store, colors):
    y_current = normal(x_vals, mean, std)
    fig.add_trace(go.Scatter(x=x_vals, y=y_current,
                             marker_color=color, 
                             name='$\sigma = {s}$'.format(s=std))
                 )
fig.update_yaxes(title_text="Probability Density")
fig.update_xaxes(title_text="x")

print("Mean: ", mean)
fig.show()

Mean:  12


From the plot, it is clear that __the standard deviation determines the _spread_ of the probability distribution and the mean determines the _centre_ location of the distribution.__ This makes sense, as the mean being a measure of central tendency, is the most likely value to occur, and the further away from it, the less likely values are to occur. The same logic can be applied to the standard deviation: if you, on average, are further away from the mean (hence a larger standard deviation), there is a larger probability of observing values further from the mean. An imporant subsection of this is known as the __standard normal distribution__, which follows the distribution $X \sim N(0, 1)$.

In [3]:
# Parameter values
mean = 0
std = 1
x_vals = np.linspace(-3, 3, 1000)
y_vals = normal(x_vals, mean, std)

# Plotting figure
fig = go.Figure()
fig.add_trace(go.Scatter(x=x_vals, y=y_vals, marker_color="orange"))
fig.update_yaxes(title_text="Probability Density")
fig.update_xaxes(title_text="x")
fig.update_layout(title_text="Standard Normal Distribution")
fig.show()

It is important to keep in mind: _"Normality is a myth; there never was, and never will be, a normal distribution" (Robert Geary, 1947)._ As we mentioned before, PDFs are only models we assume about our variables to try and compute given probabilities. Nevertheless, it may still be extremely useful. For instance, if a continuous random variable has higher probabilities near its mean and lower probabilities away from the mean, we may be able to _approximate_ the underlying distribution as normal. There are multiple methods out there to verify how well we can model a variable with a normal distribution. We will not go over them in the scope of this chapter, but a simple way of verifying this is through the use of a histogram of the sampled data. As expected, the more data and more bins we use, the better we can verify this. 

In the previous example of the measured height of students, when we assumed we were able to gather 100,000 samples, we got the following histograms:

In [4]:
x = class_heights(100000)

In [5]:
utils.visualize_density_hist(x)

In this (ideal) case we can see that our continuous variable seems to be well-modelled by a normal distribution.

# Central Limit theorem

Another, perhaps even more useful property of the normal distribution comes about through what is known as the __central limit theorem (CLT).__ The CLT formally states that, if we have $n$ observations, given that all observations were generated from the same distribution, the mean of these observations will follow (approximately) a normal distribution given that $n$ is large enough. The larger the number of observations, the better the approximation.

$$\bar{X} = \frac{1}{n}\sum_{i=1}^{n}X_{i}$$

Where $X_{1},...,X_{n}$ are the $n$ observations and $\bar{X}$ is their mean.

A common rule thumb is that the CLT can be applied when $n>35$. Not only will it have a normal distribution, but this distribution will have the following properties:

$$\bar{X} \sim N(\mu, \frac{\sigma^{2}}{n})$$

How does this actually work? Well, if we assume that all our random variables follow the same, identical distribution, then they must have the same mean. For the example of the heights of students, we can treat each student as an observation following the same, unknown distribution. However, in practice, we will never have infinitely many data points, and rarely even have 100,000. This means that depending on which students we measure the height from, __we only have an estimate of the true (population) mean__, which has its own mean and standard deviation.

For example, let us look once more at the exponential distribution we looked at above. Below, we sample 20 points from a variable with this distribution and then take the mean. This process is carried out 5 times.

In [6]:
# Sampling an exponential distribution
n = 20
for i in range(5):
    x_sample = np.random.exponential(size=n)
    print(np.mean(x_sample))

1.3355484159643647
1.176083889427009
0.9123077975795283
1.3891236152510378
1.4357608461581182


As we see from here, even though each set of 20 samples follows the same exponential distribution, the mean shows fluctuactions across the different sets, which means it must have a __standard deviation__. How does this standard deviation change with respect to the number of data points in each set we average? We check this out below for 4 different numbers of samples.

In [7]:
n_vals = [5, 10, 20, 100]

for n in n_vals:
    mean_set = np.array([])
    for i in range(1000):
        x_sample = np.random.exponential(size=n)
        mean_set = np.append(mean_set, np.mean(x_sample))
    print("n:", n, ", std:", np.std(mean_set))

n: 5 , std: 0.4336419656039068
n: 10 , std: 0.3212546858728541
n: 20 , std: 0.21676281991906812
n: 100 , std: 0.09762302618715231


As we have seen before, the more data we have, the better our estimates become, which means the more data we have, the smaller the variance and standard deviation of the estimate become, justifying why the variance of the calculated sample mean is scaled by $\frac{1}{n}$. But how do we know it follows a normal distribution if n is large enough? We will show this for the exponential distribution above via simulations.

Let us say that we collect $n$ samples from a variable that follows the exponential distribution, and find the mean of these samples. Let us also carry out this procedure 1000 more times. We now have 1000 sample means, which we can use to construct histogram estimates of the underlying distribution as we saw before! Let us do this below for different values of n:

In [8]:
# Getting data points to plot the given function
def exp_pdf(x):
    return np.exp(-x) ##
x_true = np.linspace(0, 5, 1000)
f_true = exp_pdf(x_true)

# Original distribution
x_true = np.linspace(0, 5, 1000)
f_true = exp_pdf(x_true)

# Sample mean distribution
n_vals = [2, 20, 50, 200]
mean_sets = np.array([])
for n in n_vals:
    mean_set = np.array([])
    for i in range(1000):
        x_sample = np.random.exponential(size=n)
        mean_set = np.append(mean_set, np.mean(x_sample))
    mean_sets = np.append(mean_sets, mean_set)

mean_sets = np.reshape(mean_sets, (4, 1000))

In [9]:
# Plotting original distribution
utils.visualize_exp_true(x_true, f_true)

True Mean: 1, True Variance: 1


In [10]:
# Plotting estimated distributions of sample mean
utils.visualize_mean_approx(n_vals, mean_sets)

As shown in the histograms above, when the number of observations is small, the estimated distribution is not a good fit, but as we increase the number of samples, our distribution appears more and more similar to a normal distribution and the associated standard deviation decreases! This is an incredibly powerful tool, as it works for __any unknown distribution.__ The CLT and the ability to aproximate certain random variables as having a normal distribution makes it incredibly useful in data science and statistics. We will cover many of its uses throughout the course.

The final question then becomes: how do we find the probability of an interval of a normal distribution? Let us go back to the example of the height of students. Since we generated it using a normal distribution, we know it has $\mu = 175cm$ and $\sigma = 4cm$. Let us find out what is the probability of selecting a student with a height greater than 180cm ($P(X > 180)$)? Below we plot the normal PDF and the interval which we are interested in:

In [13]:
# student height distribution
def student_normal(x):
    return normal(x, 175, 4)

def class_heights(n):
    return np.random.normal(175, 4, n)

n = 100000
bw = 0.1
interval_floor = 180
interval_ceil = 200
utils.visualize_normal_interval(interval_floor, interval_ceil, student_normal, bw=0.1, mean=175, std=4)

Since the PDF tells us the probability density at every point (tiny intervals), the probability is given by the area under the curve of that interval. We can use the sum of many tiny bars from our histogram to approximate this probability, which is what our function _prob\_estimate(  )_ does. Below, we get the appropriate probability.

In [14]:
def prob_estimate(x_lower, x_upper, pdf_func, bin_width):
    area_count = 0
    current_x = x_lower
    while current_x < x_upper:
        area_count+= pdf_func(current_x)*bin_width ## adding the area of the current interval
        current_x += bin_width ##
    return area_count

print(prob_estimate(interval_floor, interval_ceil, student_normal, bw))

0.10794477843955366


Which means we only expect about 10.8% of students to be taller than 180cm. We can approximate this from our sample by dividing the number of students taller than 180cm by the total number students, as shown below:

In [15]:
tall_count = 0
for x_point in x_sample:
    if x_point > 180:
        tall_count += 1 ##
print(tall_count/len(x_sample))

0.0


# Standardization

The normal distribution is a bell-shaped curve centered around the mean, whose width is determined by its standard deviation. It is a useful distribution, as we can approximate many random variables to have a normal distribution, and the _central limit theorem_ we previously encountered also allows us to make useful approximations.

We often use the normal distribution to give us measures of probability. However, we tend to simplify the normal distribution to only have the necessary components. So, what parts of the normal distribution actually contribute to the probability we compute? The first thing we need to understand is that the location of the mean is irrelevant. __We only care about the displacement away from the mean__. Secondly, we see from the above plots that the standard deviation affects the probability of a given displacement from the mean, meaning that __the probability depends on the displacement away from the mean proportional to the standard deviation.__

This gives us motivation for an incredibly powerful tool, known as __standardization__. The goal of standardization is to compare any normally distributed data to the __standard normal distribution__, which is a normal distribution with a population mean $\mu = 0$ and a population standard deviation $\sigma = 1$. This is useful since we don't need the explicit mean and standard deviation, but rather _how many standard deviations away from the mean a value is_ to calculate its probability.

To standardize a dataset, we carry out the following two steps:
- Subtract the mean from all points in our dataset to ensure it is centered at 0
- Divide the result by the standard deviation of the dataset, scaling the results so that the standard deviation is 1

For each data value we standardize, we denote it as shown below:

$$z = \frac{x-\mu}{\sigma}$$

Where $x$ is our data point, referred to as a __raw score__, and $z$ is known as the __z-score__ or __standard score__. By definition, it is a measure of how many standard deviations a data point is from the mean.

Why do we bother using the standard normal distribution? Couldn't we get our probabilities from the _raw_ distributions? In practice, you _could_ just use the original normal distribution, even in a statistical test. But by using z-scores, we can use _one_ hollistic distribution to describe the probabilities of _any_ normally distributed datasets. Another reason is that since to get a probability from a pdf, we have to find the area under its curve within a given range, we would to carry out complex operations for each distribution. Instead, statisticians have created a __z-table__, which tells us the _cumulative probability_ of any given z-score. We can use this to compute probabilities for any given normal distribution! While this is not _as_ relevant as when we couldn't use special calculators and computers to give us these probabilities, it still greatly simplifies the process.

Let us consider the example that we covered in the Prelude week for the height of students, denoted by $X$, where $X \sim N(175, 4^{2})$. We saw that for any continuous probability distribution, we can approximate the probability from a PDF with a histogram with very small bin widths. We used it to calculate the probability that $P(X > 180cm)$. Let us have a refresher what this looks like and what the result was.

In [16]:
import numpy as np
import plotly.graph_objects as go
import utils

def student_normal(x):
    mean, std = 175, 4
    exponent = ((x-mean)/std)**2
    exponent *= -0.5
    output = np.exp(exponent)/(std*np.sqrt(2*np.pi))
    return output

In [19]:
print(utils.prob_estimate(x_lower=180, x_upper=196, pdf_func=student_normal, bin_width=0.001))
utils.visualize_normal_interval(180, 196, student_normal, bw=0.1, mean=175, std=4)

0.1056725298899234


How could we use the standard normal distribution to get the same answer?

We can standardize our interval, such that:

$$P(X>x) \implies P(Z>\frac{x-\mu}{\sigma})$$
$$P(X>180) \implies P(Z > \frac{180-175}{4}) = P(Z > 1.25)$$

Where 1.25 is the respective _z-score_ or _standard score_ of the raw score of 180cm. We can now apply the same method as we have done above.

In [20]:
def standard_normal(x):
    mean, std = 0, 1
    exponent = ((x-mean)/std)**2
    exponent *= -0.5
    output = np.exp(exponent)/(std*np.sqrt(2*np.pi))
    return output

In [23]:
print(utils.prob_estimate(x_lower=1.25, x_upper=4, pdf_func=standard_normal, bin_width=0.001))
utils.visualize_normal_interval(1.25, 4, standard_normal, bw=0.05, mean=0, std=1)

0.1057095128641779


So we get the same probabilities in both cases! But how does this work? Well since the factors that affect the probability are the displacement from the mean proportionally to the size of the standard deviation, __the standard normal distribution contains all the necessary information given we have the approapriate z-scores.__ This means for any given interval of any normal distribution, we can always use standardization to look at different normal distributions with the same framework. Another reason why it is useful to have one hollistic distritbution, is that we can look-up __z-tables.__ 

These look-up tables contain the __cumulative probability__ of the standard normal distribution for any given _z-score_ ($\Phi(z)$). The cumulative probability tells us the probability $P(-\infty < Z < z)$ = $P(Z < z)$, and using either one or multiple cumulative probabilities from our z-table, we can calculate the probability of any interval from any normal dsitribution! Below we show what cumulative probabilities mean with a visualization for different z-scores.

In [27]:
z_list = [-3, 0, 1.25, 3, 4]

for z in z_list:
    prob = utils.prob_estimate(x_lower=-4, x_upper=z, pdf_func=standard_normal, bin_width=0.001)
    print("cumulative_probability(z={z_val}) = {probability}".format(z_val=z, probability=prob))
    utils.visualize_normal_interval(interval_floor=-4, interval_ceil=z, pdf_func=standard_normal, mean=0, std=1, bw=0.05)

cumulative_probability(z=-3) = 0.0013205106924674096


cumulative_probability(z=0) = 0.500167866768756


cumulative_probability(z=1.25) = 0.8944099274785217


cumulative_probability(z=3) = 0.9986207124133178


cumulative_probability(z=4) = 0.9999367912573766


We can confirm these results with the convential z-tables available online, such as [here](http://www.z-table.com/). Now we can see a third method for finding the probability $P(X>180)$. The method is as follows:

$$P(X>180) = P(Z>1.25) $$
$$ = 1 - P(Z<1.25) $$
$$ = 1 - \Phi(1.25) $$
$$ = 1 - 0.8944 = 0.1056$$